## Подготовка окружения

!git clone https://github.com/podmabsterio/dla_avss.git
%cd dla_avss
!pip install -r requirements.txt

## Загрузка весов

In [ ]:
out_dir = "weights" # you can change dir if you want
!bash scripts/download_convtasnet.sh $out_dir
!bash scripts/download_rtfsnet.sh $out_dir
!bash scripts/download_video_encoder.sh $out_dir

## Пример инференса

Предположим ваш датасет имеет ground_truth в таком случае вы можете сразу на инференсе посчитать все необходимые метрики указав конфигурацию с ними в metrics. Также нужно убедиться что expect_target вашего датасета выставлен в True. В случае если ваш датасет не разделен на поддатасеты следует передать partition=None.

In [ ]:
import shutil, os
if os.path.exists("predictions"):
    shutil.rmtree("predictions")

!python inference.py \
    -cn=inf_rtfsnet.yaml \
    metrics=pit \
    datasets.inf.partition=train \
    datasets.inf.expect_target=True \
    datasets.inf.dataset_path=example_data \
    video_encoder.dataset_path=example_data \
    inferencer.save_path=predictions \
    inferencer.from_pretrained=weights/rtfsnet.pth

Если же ваш датасет содержит только миксы, посчитать метрики не получится и нужно передать конфиг empty c пустым списком метрик, а также выставить expect_target=False.

In [ ]:
if os.path.exists("predictions"):
    shutil.rmtree("predictions")

!python inference.py \
    -cn=inf_rtfsnet.yaml \
    metrics=empty \
    datasets.inf.partition=inf \
    datasets.inf.expect_target=False \
    datasets.inf.dataset_path=example_data \
    video_encoder.dataset_path=example_data \
    inferencer.save_path=predictions \
    inferencer.from_pretrained=weights/rtfsnet.pth

В случае если вы уже сохранили предсказания на диск и нашли ground truth к ним и захотели посчитать метрики можно воспользоваться скриптом calc_metrics.py.

In [ ]:
!python calc_metrics.py \
    -cn=calc_metrics_rtfsnet.yaml \
    metric_calculator.pred_path=predictions/inf \
    metric_calculator.gt_path=example_data/audio/train

## Загрузка пользовательского датасета

In [ ]:
import os

dataset_link = input("Введите ссылку на ваш датасет (Google Drive / public link): ")

!gdown --folder "$dataset_link" -O data/

downloaded_dirs = [d for d in os.listdir("data") if os.path.isdir(os.path.join("data", d))]
downloaded_dirs.sort(key=lambda x: os.path.getmtime(os.path.join("data", x)), reverse=True)
dataset_dir = os.path.join("data", downloaded_dirs[0]) if downloaded_dirs else None

print(f"Найден загруженный датасет: {dataset_dir}")

In [ ]:
if os.path.isdir(os.path.join(dataset_dir, "audio", "s1")):
    expect_target=True
    metrics_conf="pit"
else:
    expect_target=False
    metrics_conf="empty"


In [ ]:
if os.path.exists("predictions"):
    shutil.rmtree("predictions")

!python inference.py \
    -cn=inf_rtfsnet.yaml \
    metrics=$metrics_conf \
    datasets.inf.partition=null \
    datasets.inf.expect_target=$expect_target \
    datasets.inf.dataset_path=$dataset_dir \
    video_encoder.dataset_path=$dataset_dir \
    inferencer.save_path=predictions \
    inferencer.from_pretrained=weights/rtfsnet.pth